# Checking the symmetry of methods

First, we need to import the packages that will be used in the following code.

In [1]:
import Pkg; Pkg.add("BSeries")
import Pkg; Pkg.add("RootedTrees")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [2]:
using BSeries
using RootedTrees

## Antipode
Since we now that the inverse element in the Butcher group is defined as $a^{-1}=a \circ S$, where S is the so-called antipode, we need to define this antipode first.

In [3]:
function antipode(t::RootedTree)
    coproduct = all_splittings(t)
    
    forests = coproduct[1]
    subtrees = coproduct[2]
    number = length(forests)
    
    coeffs = [-1]
    trees = [[t]]

    for i in 2:(number-1)

        leftovertree = subtrees[i]
        
        for m in antipode(leftovertree)[2]
            empty2 = []
            #length = length(m)

            for tree in forests[i]
            push!(empty2, tree)
            end
            
            for n in m
                push!(empty2, n)
            end
            
            push!(trees, empty2)
        end

        for j in antipode(leftovertree)[1]
            push!(coeffs, -1*j)
        end
      
    end   
        
        
    
    return [coeffs, trees]
end

antipode (generic function with 1 method)

Now we are using the antipode for the inverse.

In [4]:
function inverse(b::TruncatedBSeries, t::RootedTree)
    result = 0
    antipode_tree = antipode(t)

    k = 1
    for i in antipode_tree[1]
        btw = i
        for j in antipode_tree[2][k]
            btw *= b[j]
        end
        k += 1
        result += btw
    end
    return result
    
end

inverse (generic function with 1 method)

Finally, we can define the adjoint of a given B-series. The function "adjoint" returns the time-reversed method of the inverse B-series.

In [5]:
function adjoint(b::TruncatedBSeries)
    series = TruncatedBSeries{RootedTree{Int, Vector{Int}}, Rational{Int64}}()

    series[rootedtree(Int[])] = one(Rational{Int64})
    orderofb = order(b)
    for o in 1:orderofb
        for t in RootedTreeIterator(o)
            series[copy(t)] = (-1)^(order(t)) * inverse(b,t)
        end
    end
    return series
end

adjoint (generic function with 1 method)

## Checking up to what order a method if symmetric

In [6]:
function SymmetricOrder(b::TruncatedBSeries)
    
    adj = adjoint(b)

    for o in 0:order(b)
        # Iterate over all rooted trees used as keys in `series`
        # of a given order `o`.
        for t in RootedTreeIterator(o)
            if (b[t] - adj[t]) != 0
                return order(t) - 1
            end
        end
    end

    return order(b)
end

SymmetricOrder (generic function with 1 method)

## Examples

### Implicit Midpoint
The implicit midpoint method is symmetric. Thus, the symmetric order will be always the same as the order we choose for the B-series.

In [7]:
A_im = [1//2 ;;]
b_im = [1]
c_im = [1//2]
rk_im = RungeKuttaMethod(A_im, b_im, c_im)
bseries_im = bseries(rk_im, 6);

In [8]:
SymmetricOrder(bseries_im)

6

### Explicit Midpoint
The Explicit Midpoint is only symmetric up to order 3.

In [9]:
A_em = [0 0; 1//2 0]
b_em = [0, 1]
c_em = [0, 1//2]
rk_em = RungeKuttaMethod(A_em, b_em, c_em)
bseries_em = bseries(rk_em, 6);

In [10]:
SymmetricOrder(bseries_em)

3

### Implicit and Explicit Euler
These two methods are adjoint???.

In [11]:
A_ex_euler = [0 ;;]
b_ex_euler = [1]
c_ex_euler = [0]
rk_ex_euler = RungeKuttaMethod(A_ex_euler, b_ex_euler, b_ex_euler)
bseries_ex_euler = bseries(rk_ex_euler, 5);

In [12]:
A_im_euler = [1 ;;]
b_im_euler = [1]
c_im_euler = [1]
rk_im_euler = RungeKuttaMethod(A_im_euler, b_im_euler, b_im_euler)
bseries_im_euler = bseries(rk_im_euler, 5);

We define the error as the difference between a B-series and the adjoint of a different B-series.
As you can see, both errors are euqal to zero. This shows again that these two methods are adjoint??.

In [13]:
bseries_ex_euler - adjoint(bseries_im_euler)

TruncatedBSeries{RootedTree{Int64, Vector{Int64}}, Rational{Int64}} with 18 entries:
  RootedTree{Int64}: Int64[]         => 0//1
  RootedTree{Int64}: [1]             => 0//1
  RootedTree{Int64}: [1, 2]          => 0//1
  RootedTree{Int64}: [1, 2, 3]       => 0//1
  RootedTree{Int64}: [1, 2, 2]       => 0//1
  RootedTree{Int64}: [1, 2, 3, 4]    => 0//1
  RootedTree{Int64}: [1, 2, 3, 3]    => 0//1
  RootedTree{Int64}: [1, 2, 3, 2]    => 0//1
  RootedTree{Int64}: [1, 2, 2, 2]    => 0//1
  RootedTree{Int64}: [1, 2, 3, 4, 5] => 0//1
  RootedTree{Int64}: [1, 2, 3, 4, 4] => 0//1
  RootedTree{Int64}: [1, 2, 3, 4, 3] => 0//1
  RootedTree{Int64}: [1, 2, 3, 4, 2] => 0//1
  RootedTree{Int64}: [1, 2, 3, 3, 3] => 0//1
  RootedTree{Int64}: [1, 2, 3, 3, 2] => 0//1
  RootedTree{Int64}: [1, 2, 3, 2, 3] => 0//1
  RootedTree{Int64}: [1, 2, 3, 2, 2] => 0//1
  RootedTree{Int64}: [1, 2, 2, 2, 2] => 0//1

In [14]:
bseries_im_euler - adjoint(bseries_ex_euler)

TruncatedBSeries{RootedTree{Int64, Vector{Int64}}, Rational{Int64}} with 18 entries:
  RootedTree{Int64}: Int64[]         => 0//1
  RootedTree{Int64}: [1]             => 0//1
  RootedTree{Int64}: [1, 2]          => 0//1
  RootedTree{Int64}: [1, 2, 3]       => 0//1
  RootedTree{Int64}: [1, 2, 2]       => 0//1
  RootedTree{Int64}: [1, 2, 3, 4]    => 0//1
  RootedTree{Int64}: [1, 2, 3, 3]    => 0//1
  RootedTree{Int64}: [1, 2, 3, 2]    => 0//1
  RootedTree{Int64}: [1, 2, 2, 2]    => 0//1
  RootedTree{Int64}: [1, 2, 3, 4, 5] => 0//1
  RootedTree{Int64}: [1, 2, 3, 4, 4] => 0//1
  RootedTree{Int64}: [1, 2, 3, 4, 3] => 0//1
  RootedTree{Int64}: [1, 2, 3, 4, 2] => 0//1
  RootedTree{Int64}: [1, 2, 3, 3, 3] => 0//1
  RootedTree{Int64}: [1, 2, 3, 3, 2] => 0//1
  RootedTree{Int64}: [1, 2, 3, 2, 3] => 0//1
  RootedTree{Int64}: [1, 2, 3, 2, 2] => 0//1
  RootedTree{Int64}: [1, 2, 2, 2, 2] => 0//1